In [1]:
from inventory import *
import requests
from bs4 import BeautifulSoup
from PIL import Image
import os.path
from os import path
from random import randrange
import numpy as np
import ast

In [2]:
# get links for all mezlan shoes
links = []
types = ['formal-dress-shoes', 'loafers', 'oxford', 'boots', 'sneakers']

for t in types:
    i = 1
    print(t)
    while True:
        result = requests.get("https://www.carrucci.com/collections/" + t + "?page="+str(i))
        print(i)
        i += 1
        soup = BeautifulSoup(result.content, "html.parser")
        a = soup.find_all("div", {"class": "ProductItem__Wrapper"})
        ln = ["https://www.carrucci.com" + str(ax.find('a')['href']) for ax in a]
        if len(ln) == 0:
            break
        links.extend(ln)
    print()

formal-dress-shoes
1
2

loafers
1
2
3
4
5
6

oxford
1
2
3
4

boots
1
2

sneakers
1
2



In [3]:
r = {}
for l in links:
    result = requests.get(l)
    soup = BeautifulSoup(result.content, "html.parser")
    title = soup.title.text.replace("\n", "").replace("  ", "").replace(" – Carrucci Shoes", "").replace("Carrucci ", "")
    r[title] = soup
    print(title)

Genuine Leather Bow Tie Loafer
KS525-210SP Bow Tie Loafer
KS525-103V Velvet Loafer
KS308-101V Velvet Buckle Loafer
KS805-10SS Studs Loafer-Black
KS525-102V, Velvet Prom Loafer
KS525-01P, Patent Leather Lace-up Oxford
KS308-01 Velvet Loafer
Prom Loafer in Leather Sole
KS805-05SG Suede Studs Loafer
KS805-07SV Hand Embroidered Loafer
KS525-210N Formal Dress Shoes
KS525-102P Patent Leather Bow Tie Dress Shoes
Buckle Loafer With Gator Print Embossed Leather
KS503-37A Hand Painted Double Monk Loafer
KS525-17S Printed Suede Double Monk Straps
KS478-505 Leather Buckle Loafer
KS515-04 Double Monk Strap Leather Loafer
KS503-02 Signature Buckle Loafer
KS479-601 Contrast Penny Loafer
KS525-305 Soft Leather Casual Buckle Loafer/Mule
KS515-03 Whole Cut Leather Loafer
KS509-23 Double Monk Straps Leather Shoes
KS478-120SC Patina Finish Leather Penny Loafer
KS478-119S Suede Penny Loafer w/ Leather Trim
KS503-60 Cross Strap Leather Loafer
KS500-12 Woven Cap Toe Tassel Loafer
KS478-118S Leather Suede Pen

In [4]:
import matplotlib.pyplot as plt

In [5]:
def saveSwatch(color, imglink):
    # finds and downloads swatches
    response = requests.get(imglink)
    try:
        image = Image.open(BytesIO(response.content))
    except:
        return
    x, y = image.size
    xl = x//2 - x//20
    xr = xl + x//10
    yl = 13*y//16 - y//20
    yr = yl + y//10
    image = image.crop((xl, yl, xr, yr))
    pt = "swatches/"+ (color.lower().replace(' ', '-').replace('/', '-'))

#     if path.exists(pt):
    image.save(pt + str(randrange(10)) +".png")
#     else:
#         image.save(pt +".png")

In [6]:
def get(soup):
    title = soup.title.text.replace("\n", "").replace("  ", "").replace(" – Carrucci Shoes", "")
#     print(title)

for i in r.keys():
    get(r[i])

soups = [i for i in r.values()]

products = []

k = 0
for soup in soups:
    k += 1
#     if k > 15: break
    # soup = soups[1]
    
    # vendor
    vendor = "Carrucci"

    # title

    title = soup.title.text.replace("\n", "").replace("  ", "").replace(" – Carrucci Shoes", "").replace("Carrucci ", "")
    title = title.replace("Navy/Olive/Cognac", "").replace("-Black", "").replace("-Emerald", "").replace("PRE ORDER", "")
    title = title.replace("-Brown/Cognac", "").replace("-Brown/Navy", "").replace("-Navy/Red", "").replace("-Brown", "")
    title = title.replace("-Whisky", "").replace("-Ocean Blue", "").replace("-Cobalt", "").replace("-Gray", "")
    title = title.replace("-Wheat", "").replace("-Burgundy/Black", "").replace("-Burgundy", "").replace(" - Blue", "")
    title = title.title()
    
    x = title.replace(' ', '').lower().replace("-", "").replace("/", "").replace(",", "").replace("carrucci", "").replace("carucci", "").replace("carruci", "")
    handle = x[:15]
    
    # type
    type = "Dress Shoe"
    
    if "Oxford" in title:
        type = "Oxford"
    elif "Loafer" in title or "Slip-On" in title or "Slip On" in title or "Driver" in title:
        type = "Loafer and Slip On"
    elif "Boot" in title:
        type="Boot"
    elif "Sneaker" in title:
        type="Dress Casual Shoe"
    else:
        type="Dress Shoe"

    # seo title
    seotitle = 'Brite Creations | ' + title + ' by ' + vendor

    # body
    body = soup.find("div", {"class": "ProductMeta__Description"}).find("div", {"class": "Rte"})
    if body.find("ul") is None:
        body = body
    else:
        body = body.find("ul")
    body = str(body).replace("<div class=\"Rte\">", "").replace("</div>", "").replace("<span>", "").replace("</span>", "")
    body = "<meta charset=\"utf-8\">\n" + body + "\n<br><p>Best quality and price from <a href=\"https://britecreations.com/pages/about\">Brite Creations Atlanta</a>.</p>"
    
#     seodescription
    seodescription = 'Shop at Brite Creations Atlanta for the best deal on ' + title + " " + type +' by ' + vendor + '\n' + body
    
#     images
    imgs = soup.find_all("img")
    images = ["https:" + i['data-src'].replace("_{width}x", "") for i in imgs if "data-widths" in str(i)]
    
    colors = soup.find("div", {"class":"ProductForm__Variants"})
    colors = colors.find_all("input", {"class" : "ColorSwatch__Radio"})
    colors = [c["value"] for c in colors]
    
    sizes = soup.find("div", {"class":"ProductForm__Variants"})
    sizes = sizes.find_all("input", {"class" : "SizeSwatch__Radio"})
    sizes = [c["value"] for c in sizes]
    
#     print(sizes)
    
    
    
    if len(colors) == 0:
        if "KS525-102P" in soup.title.text:
            colors = ['Black']
        if "KS500-12" in soup.title.text:
            colors = ["Cognac"]
        if "KS524-12" in soup.title.text:
            colors = sizes[:4]
            sizes = sizes[4:]
        if "KS886-734" in soup.title.text:
            colors = sizes[:2]
            sizes = sizes[2:]
    if "KB2019-11" in soup.title.text:
        colors[0] = "Brown"
    
    ncols = []
    for c in colors:
        if c not in ncols:
            ncols.append(c)
    colors = ncols
            
    fcolors = []
    for i in images:
        for c in colors:
            if c.replace("/", "").split()[0] in i or c.replace("/", "_").split()[0] in i or c.replace("/", " ").split()[0] in i:
                if c not in [a for a, b in fcolors]:
                    fcolors.append((c, i))
                break
    
    if len(fcolors) == 0:
        if len(images) == 1:
            for c in colors:
                fcolors.append((c, images[0]))
        if len(images) == len(colors):
            for c in range(len(colors)):
                fcolors.append((colors[c], images[c]))
        if len(colors) == 1:
            fcolors.append((colors[0], images[0]))
        if "KS503-37A" in soup.title.text:
            i = 2
            for c in colors:
                fcolors.append((c, images[i]))
                i += 1
        if "KS673-01" in soup.title.text:
            i = 1
            for c in colors:
                fcolors.append((c, images[i - 1]))
                i *= 2
                
    if len(colors) != len(fcolors):
        if len(images) == len(colors):
            fcolors = []
            for c in range(len(colors)):
                fcolors.append((colors[c], images[c]))

    if len(colors) != len(fcolors) and len(fcolors) != len(images):
        continue
        # print(title)
        # print(fcolors)
        # print(colors)
        
        # fig = plt.figure(figsize=(10, 10))
        
        # Cols = 3
        # Rows = len(images) // Cols 
        # Rows += len(images) % Cols
        
        # for i in range(len(images)):
        #     print(i, images[i])
        #     f = requests.get(images[i])
        #     im = Image.open(BytesIO(f.content))
        #     ax = fig.add_subplot(Rows,Cols, i+1)
        #     ax.set_title(i)
        #     ax.axis("off")
        #     ax.imshow(im)
            
        
        # plt.tight_layout()
        # plt.show()
        
        # for i in colors:
        #     text = "Enter index of image for " + i + " : "
        #     img = input(text)
        #     if int(img) == -1:
        #         img = ""
        #     else:
        #         img = images[int(img)]
        #     fcolors.append((c, img))
        # print(fcolors)

    colors = fcolors
#     print(colors)
#     print()

    images = [(i, "") for i in images]

        #     tags
    tags = []
    for c, i in colors:
        tag = 'color:' + c
        if tag not in tags:
            tags.append(tag)
    for s in sizes:
        tag = 'size:' + str(s).replace('.0', '')
        if tag not in tags:
            tags.append(tag)

#     print(tags)
        
    #     price
    p = soup.find_all("div", {"class": "ProductMeta__PriceList Heading"})
    pr = p[0].find_all("span")[0].text[1:]
    op = p[0].find_all("span")[-1].text[1:]
    p = 95 * float(pr) /100
    
    price = []
    oldprice = []
    for i in range(len(colors) * len(sizes)):
        price.append(p)
        oldprice.append(op)
    
#     inventory and sku
    inventory = []
    sku = []
    
    for i in range(len(colors) * len(sizes)):
        inventory.append((1, 0)) #(online, retail)
    
    for s in sizes:
        for c, i in colors:
                sku.append(handle+"-"+c.lower()+"-"+str(s).replace('.0', ''))
    
#     swatches
    for c, i in colors:
        if "http" in i:
            saveSwatch(c, i)
    
    prod = product(handle=handle, title=title, body=body, vendor=vendor, type=type, sizes=sizes, 
                   colors=colors, images=images, tags=tags, price=price, oldprice=oldprice, seotitle=seotitle, 
                   seodescription=seodescription, sku=sku, inventory=inventory)
    products.append(prod)

#     break

In [7]:
productstocsv(products)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
Errors: []


In [30]:
productstoinventory(products)

In [25]:
import numpy as np

In [ ]:
for i in r.keys():
    get(r[i])

soups = [i for i in r.values()]

products = []

k = 0
for soup in soups:
    
#     if k > 15: break
    # soup = soups[1]
    
    # vendor
    vendor = "Carrucci"

    # title

    title = soup.title.text.replace("\n", "").replace("  ", "").replace(" – Carrucci Shoes", "").replace("Carrucci ", "")
    title = title.replace("Navy/Olive/Cognac", "").replace("-Black", "").replace("-Emerald", "").replace("PRE ORDER", "")
    title = title.replace("-Brown/Cognac", "").replace("-Brown/Navy", "").replace("-Navy/Red", "").replace("-Brown", "")
    title = title.replace("-Whisky", "").replace("-Ocean Blue", "").replace("-Cobalt", "").replace("-Gray", "")
    title = title.replace("-Wheat", "").replace("-Burgundy/Black", "").replace("-Burgundy", "").replace(" - Blue", "")
    title = title.title()
    
    x = title.replace(' ', '').lower().replace("-", "").replace("/", "").replace(",", "").replace("carrucci", "").replace("carucci", "").replace("carruci", "")
    handle = str(np.random.randint(1000)) + x[:15]
    products[k].handle = handle
    k += 1